<a href="https://colab.research.google.com/github/Chienstartup/ISIC_2024/blob/main/features_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00


In [10]:
import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold,StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import PredefinedSplit
from tqdm import tqdm

In [4]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/df_train.csv')

<h3>Split dataset into 5 Folds</h3>

In [5]:
N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)
df_train["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx

In [8]:
feature_cols = [x for x in df_train.columns if x not in ["target", "fold", "patient_id", 'isic_id',]]

In [11]:
def custom_metric_binary(y_true, y_pred):
    y_hat = y_pred[:, 1] if y_pred.ndim > 1 else y_pred
    y_true_binary = y_true
    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    v_gt = 1 - y_true_binary
    v_pred = 1 - y_hat
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc

In [12]:
def optimize_feature_selection(sampled_df, feature_cols, target_col, n_trials=10):
    X = sampled_df[feature_cols]
    y = sampled_df[target_col]
    fold = sampled_df['fold']

    best_features = []
    best_overall_score = -np.inf
    remaining_features = feature_cols.copy()

    while remaining_features:
        best_score = -np.inf
        best_feature = None

        for feature in tqdm(remaining_features, desc=f"Evaluating features (current best: {len(best_features)})"):
            current_features = best_features + [feature]

            study = optuna.create_study(direction='maximize')
            scores = []

            for _ in range(n_trials):
                study.optimize(lambda trial: lgb_objective(trial, X, y, fold, current_features), n_trials=1)
                scores.append(study.best_value)

            avg_score = np.mean(scores)

            if avg_score > best_score:
                best_score = avg_score
                best_feature = feature

        if best_score > best_overall_score:
            best_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_overall_score = best_score
            print(f"\nAdded feature: {best_feature}")
            print(f"New best score: {best_score:.4f}")
            print("Current feature set:")
            for i, feat in enumerate(best_features, 1):
                print(f"{i}. {feat}")
        else:
            print(f"\nStopping: No improvement (best overall: {best_overall_score:.4f}, current best: {best_score:.4f})")
            break

    print(f"\nFinal best feature combination:")
    for i, feature in enumerate(best_features, 1):
        print(f"{i}. {feature}")
    print(f"Best overall score: {best_overall_score:.4f}")

    return best_features

def lgb_objective(trial, X, y, fold, selected_features):
    params = {
        'objective': 'binary',
        'num_class': 1,
        'n_estimators': 200,
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'num_leaves': trial.suggest_int('num_leaves', 16, 256),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.8, 4.0),
        'device': 'cpu'
    }

    ps = PredefinedSplit(fold)

    cv_scores = []
    for train_index, val_index in ps.split():
        X_train, X_val = X.iloc[train_index][selected_features], X.iloc[val_index][selected_features]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = lgb.LGBMClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict_proba(X_val)
        score = custom_metric_binary(y_val, y_pred)
        cv_scores.append(score)

    cv_score = np.mean(cv_scores)
    return cv_score


In [13]:
best_features = optimize_feature_selection(df_train, feature_cols, 'target', n_trials=1)
print("Best feature combination:", best_features)

Evaluating features (current best: 0):   0%|          | 0/124 [00:00<?, ?it/s][I 2024-09-11 08:35:06,226] A new study created in memory with name: no-name-c3365a5f-9447-44d9-a29a-c224a45b0118
[I 2024-09-11 08:35:06,968] Trial 0 finished with value: 0.03088223288233165 and parameters: {'lambda_l1': 5.294003892553125, 'lambda_l2': 0.07408247868285786, 'learning_rate': 0.06168355492491374, 'max_depth': 8, 'num_leaves': 211, 'colsample_bytree': 0.8402081749780288, 'colsample_bynode': 0.8817645764858415, 'bagging_fraction': 0.9263357375313878, 'bagging_freq': 1, 'min_child_samples': 14, 'scale_pos_weight': 3.0033960674544344}. Best is trial 0 with value: 0.03088223288233165.
Evaluating features (current best: 0):   1%|          | 1/124 [00:00<01:31,  1.34it/s][I 2024-09-11 08:35:06,979] A new study created in memory with name: no-name-e34b2e0e-bb49-42f2-9166-edb43851be4e
[I 2024-09-11 08:35:07,479] Trial 0 finished with value: 0.090252952433932 and parameters: {'lambda_l1': 0.73936055445468


Added feature: clin_size_long_diam_mm
New best score: 0.0903
Current feature set:
1. clin_size_long_diam_mm


Evaluating features (current best: 1):   0%|          | 0/123 [00:00<?, ?it/s][I 2024-09-11 08:36:43,261] A new study created in memory with name: no-name-918230e9-1a9b-4cde-83bf-6f95b0c6b677
[I 2024-09-11 08:36:43,893] Trial 0 finished with value: 0.10265085375095244 and parameters: {'lambda_l1': 0.0020198805325433006, 'lambda_l2': 0.03871505168236007, 'learning_rate': 0.08949626435895061, 'max_depth': 4, 'num_leaves': 31, 'colsample_bytree': 0.8561731674833717, 'colsample_bynode': 0.7006337279115988, 'bagging_fraction': 0.8378134391937835, 'bagging_freq': 2, 'min_child_samples': 47, 'scale_pos_weight': 3.248569531919732}. Best is trial 0 with value: 0.10265085375095244.
Evaluating features (current best: 1):   1%|          | 1/123 [00:00<01:17,  1.58it/s][I 2024-09-11 08:36:43,898] A new study created in memory with name: no-name-05147753-1051-4322-9438-3fc6bf3816d4
[I 2024-09-11 08:36:44,698] Trial 0 finished with value: 0.10161670786997926 and parameters: {'lambda_l1': 0.0986713890


Added feature: tbp_lv_H_between_std
New best score: 0.1265
Current feature set:
1. clin_size_long_diam_mm
2. tbp_lv_H_between_std


Evaluating features (current best: 2):   0%|          | 0/122 [00:00<?, ?it/s][I 2024-09-11 08:38:22,262] A new study created in memory with name: no-name-40718e97-6378-4fe1-b07c-42918aeb13d5
[I 2024-09-11 08:38:23,330] Trial 0 finished with value: 0.1227718162399793 and parameters: {'lambda_l1': 0.07284983267431701, 'lambda_l2': 0.023711464859400223, 'learning_rate': 0.015517867720591619, 'max_depth': 7, 'num_leaves': 231, 'colsample_bytree': 0.8220437647653114, 'colsample_bynode': 0.6856771192083685, 'bagging_fraction': 0.8993941168224606, 'bagging_freq': 6, 'min_child_samples': 28, 'scale_pos_weight': 1.783994337710762}. Best is trial 0 with value: 0.1227718162399793.
Evaluating features (current best: 2):   1%|          | 1/122 [00:01<02:09,  1.07s/it][I 2024-09-11 08:38:23,335] A new study created in memory with name: no-name-3f44d339-6831-49f9-ae83-83dd34b585ee
[I 2024-09-11 08:38:24,309] Trial 0 finished with value: 0.12192232784966044 and parameters: {'lambda_l1': 0.00277186763

...


Added feature: tbp_lv_B_between_std
New best score: 0.1677
Current feature set:
1. clin_size_long_diam_mm
2. tbp_lv_H_between_std
3. lesion_visibility_score
4. tbp_lv_deltaLBnorm_between_std
5. age_approx_zscore
6. tbp_lv_B
7. avg_contrast
8. hue_contrast
9. anatom_site_general_encoded
10. attribution_encoded
11. tbp_lv_C
12. tbp_lv_nevi_confidence_between_std
13. tbp_lv_B_between_std


Evaluating features (current best: 13):   0%|          | 0/111 [00:00<?, ?it/s][I 2024-09-11 09:01:43,961] A new study created in memory with name: no-name-55a064c2-572d-4445-a0bd-55d3915b56d3
[I 2024-09-11 09:01:45,354] Trial 0 finished with value: 0.16496232367842525 and parameters: {'lambda_l1': 6.763624554992923, 'lambda_l2': 0.02104668273617002, 'learning_rate': 0.04425678538967845, 'max_depth': 7, 'num_leaves': 144, 'colsample_bytree': 0.83848914945385, 'colsample_bynode': 0.7416508078079678, 'bagging_fraction': 0.4942582550581113, 'bagging_freq': 1, 'min_child_samples': 21, 'scale_pos_weight': 2.8986701520642546}. Best is trial 0 with value: 0.16496232367842525.
Evaluating features (current best: 13):   1%|          | 1/111 [00:01<02:33,  1.40s/it][I 2024-09-11 09:01:45,357] A new study created in memory with name: no-name-14d39080-4f46-4cab-85ce-488af82d857f
[I 2024-09-11 09:01:46,587] Trial 0 finished with value: 0.16020431499433796 and parameters: {'lambda_l1': 4.007240273324


Added feature: tbp_lv_Aext
New best score: 0.1680
Current feature set:
1. clin_size_long_diam_mm
2. tbp_lv_H_between_std
3. lesion_visibility_score
4. tbp_lv_deltaLBnorm_between_std
5. age_approx_zscore
6. tbp_lv_B
7. avg_contrast
8. hue_contrast
9. anatom_site_general_encoded
10. attribution_encoded
11. tbp_lv_C
12. tbp_lv_nevi_confidence_between_std
13. tbp_lv_B_between_std
14. tbp_lv_Aext


Evaluating features (current best: 14):   0%|          | 0/110 [00:00<?, ?it/s][I 2024-09-11 09:04:11,377] A new study created in memory with name: no-name-adb8aa61-39c7-4e98-8f01-623d73c87b8c
[I 2024-09-11 09:04:12,543] Trial 0 finished with value: 0.15599478242640724 and parameters: {'lambda_l1': 0.23120114959191834, 'lambda_l2': 4.222396436327102, 'learning_rate': 0.01284066191587296, 'max_depth': 5, 'num_leaves': 62, 'colsample_bytree': 0.9942596559953474, 'colsample_bynode': 0.42695783340289495, 'bagging_fraction': 0.9531602297263652, 'bagging_freq': 1, 'min_child_samples': 83, 'scale_pos_weight': 1.2812268995366864}. Best is trial 0 with value: 0.15599478242640724.
Evaluating features (current best: 14):   1%|          | 1/110 [00:01<02:07,  1.17s/it][I 2024-09-11 09:04:12,551] A new study created in memory with name: no-name-82d117f0-ef90-43db-9a42-469ef4bcfa05
[I 2024-09-11 09:04:13,452] Trial 0 finished with value: 0.16274327421531035 and parameters: {'lambda_l1': 0.0984187854


Stopping: No improvement (best overall: 0.1680, current best: 0.1665)

Final best feature combination:
1. clin_size_long_diam_mm
2. tbp_lv_H_between_std
3. lesion_visibility_score
4. tbp_lv_deltaLBnorm_between_std
5. age_approx_zscore
6. tbp_lv_B
7. avg_contrast
8. hue_contrast
9. anatom_site_general_encoded
10. attribution_encoded
11. tbp_lv_C
12. tbp_lv_nevi_confidence_between_std
13. tbp_lv_B_between_std
14. tbp_lv_Aext
Best overall score: 0.1680
Best feature combination: ['clin_size_long_diam_mm', 'tbp_lv_H_between_std', 'lesion_visibility_score', 'tbp_lv_deltaLBnorm_between_std', 'age_approx_zscore', 'tbp_lv_B', 'avg_contrast', 'hue_contrast', 'anatom_site_general_encoded', 'attribution_encoded', 'tbp_lv_C', 'tbp_lv_nevi_confidence_between_std', 'tbp_lv_B_between_std', 'tbp_lv_Aext']


In [14]:
df_train[best_features].to_csv('df_train_filter.csv', index = False)